In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
259,AFG,Asia,Afghanistan,2020-09-15,38815.0,99.0,45.857,1426.0,6.0,1.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
450,ALB,Europe,Albania,2020-09-15,11520.0,167.0,159.143,338.0,4.0,2.714,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
710,DZA,Africa,Algeria,2020-09-15,48496.0,242.0,263.286,1620.0,8.0,8.286,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-14,France,390412.0,0.0,30949.0,0.0,0.0,0.0
0,2020-09-14,China,90225.0,0.0,4739.0,0.0,0.0,0.0
0,2020-09-14,Italy,289085.0,0.0,35635.0,0.0,0.0,0.0
0,2020-09-14,Spain,568836.0,0.0,29754.0,0.0,0.0,0.0
0,2020-09-14,United States,6600580.0,0.0,195220.0,0.0,0.0,0.0
0,2020-09-14,World,29573293.0,0.0,934276.0,0.0,0.0,0.0
0,2020-09-14,United Kingdom,371234.0,0.0,41669.0,0.0,0.0,0.0
0,2020-09-14,Germany,261296.0,0.0,9357.0,0.0,0.0,0.0
0,2020-09-14,Iran,405873.0,0.0,23391.0,0.0,0.0,0.0
0,2020-09-14,Turkey,292387.0,0.0,7094.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-15,France,390412.0,387252.0,30949.0,30950.0,3160.0,-1.0
0,2020-09-15,China,90225.0,90219.0,4739.0,4735.0,6.0,4.0
0,2020-09-15,Italy,289085.0,288761.0,35635.0,35624.0,324.0,11.0
0,2020-09-14,Spain,568836.0,593730.0,29754.0,29848.0,-24894.0,-94.0
0,2020-09-15,United States,6600580.0,6554820.0,195220.0,194530.0,45760.0,690.0
0,2020-09-15,World,29573293.0,29309546.0,934276.0,928890.0,263747.0,5386.0
0,2020-09-15,United Kingdom,371234.0,371125.0,41669.0,41637.0,109.0,32.0
0,2020-09-15,Germany,261296.0,261762.0,9357.0,9362.0,-466.0,-5.0
0,2020-09-15,Iran,405873.0,404648.0,23391.0,23313.0,1225.0,78.0
0,2020-09-15,Turkey,292387.0,292878.0,7094.0,7119.0,-491.0,-25.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")